# 175. Combine Two Tables
Write a solution to report the first name, last name, city, and state of each person in the Person table. If the address of a personId is not present in the Address table, report null instead.

In [3]:
import pandas as pd
import datetime as data

data = [[1, 'Wang', 'Allen'], [2, 'Alice', 'Bob']]
person = pd.DataFrame(data, columns=['personId', 'firstName', 'lastName']).astype({'personId':'Int64', 'firstName':'object', 'lastName':'object'})
data = [[1, 2, 'New York City', 'New York'], [2, 3, 'Leetcode', 'California']]
address = pd.DataFrame(data, columns=['addressId', 'personId', 'city', 'state']).astype({'addressId':'Int64', 'personId':'Int64', 'city':'object', 'state':'object'})

print(person)
print(address)

   personId firstName lastName
0         1      Wang    Allen
1         2     Alice      Bob
   addressId  personId           city       state
0          1         2  New York City    New York
1          2         3       Leetcode  California


In [ ]:
import pandas as pd

def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    df = person.merge(address, how='left', on='personId') #相當於SQL的left join
    result = df[['firstName','lastName','city', 'state']]
    return result

# print(combine_two_tables(person,address))

  firstName lastName           city     state
0      Wang    Allen            NaN       NaN
1     Alice      Bob  New York City  New York


# 181. Employees Earning More Than Their Managers

Write a solution to find the employees who earn more than their managers.

In [2]:
data = [[1, 'Joe', 70000, 3], [2, 'Henry', 80000, 4], [3, 'Sam', 60000, None], [4, 'Max', 90000, None]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'managerId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'managerId':'Int64'})

print(employee)

   id   name  salary  managerId
0   1    Joe   70000          3
1   2  Henry   80000          4
2   3    Sam   60000       <NA>
3   4    Max   90000       <NA>


In [3]:
import pandas as pd

def find_employees(employee: pd.DataFrame) -> pd.DataFrame:
    # 合併表格，使用managerId和id作為join keys
    merge_df = pd.merge(employee, employee, left_on='managerId', right_on='id', how='inner', suffixes=('_Employee', '_Manager')) #left_on：左表的連接鍵為managerId, right_on右表的連接鍵為id
    # return merge_df

    #篩選條件：經理salary < employee salary
    result_df = merge_df[merge_df['salary_Employee'] > merge_df['salary_Manager']]

    #最終：選取員工姓名並重新命名
    final_result = result_df[['name_Employee']].rename(columns={'name_Employee':'Employee'})

    return final_result
print(find_employees(employee))

  Employee
0      Joe


# 1581. Customer Who Visited but Did Not Make Any Transactions

Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

In [ ]:
data = [[1, 23], [2, 9], [4, 30], [5, 54], [6, 96], [7, 54], [8, 54]]
visits = pd.DataFrame(data, columns=['visit_id', 'customer_id']).astype({'visit_id':'Int64', 'customer_id':'Int64'})
data = [[2, 5, 310], [3, 5, 300], [9, 5, 200], [12, 1, 910], [13, 2, 970]]
transactions = pd.DataFrame(data, columns=['transaction_id', 'visit_id', 'amount']).astype({'transaction_id':'Int64', 'visit_id':'Int64', 'amount':'Int64'})

print(visits)
print(transactions)

   visit_id  customer_id
0         1           23
1         2            9
2         4           30
3         5           54
4         6           96
5         7           54
6         8           54
   transaction_id  visit_id  amount
0               2         5     310
1               3         5     300
2               9         5     200
3              12         1     910
4              13         2     970


In [ ]:
import pandas as pd
merge_df = visits.merge(transactions, how = 'left', left_on='visit_id', right_on='visit_id', suffixes=('_visits', "_transactions" )) #左表右表的連接鍵
no_trans = merge_df[merge_df['transaction_id'].isnull()]
result = no_trans.groupby('customer_id').size().reset_index(name='count_no_trans').sort_values('count_no_trans', ascending=False) #groupny之後，customer_id=index,Series為size計數，所以才需要reset_index把它變為dataframe
print(result)


   customer_id  count_no_trans
1           54               2
0           30               1
2           96               1


# 1587. Bank Account Summary II
### Write a solution to report the name and balance of users with a balance higher than 10000. The balance of an account is equal to the sum of the amounts of all transactions involving that account.

In [29]:
data = [[900001, 'Alice'], [900002, 'Bob'], [900003, 'Charlie']]
users = pd.DataFrame(data, columns=['account', 'name']).astype({'account':'Int64', 'name':'object'})
data = [[1, 900001, 7000, '2020-08-01'], [2, 900001, 7000, '2020-09-01'], [3, 900001, -3000, '2020-09-02'], [4, 900002, 1000, '2020-09-12'], [5, 900003, 6000, '2020-08-07'], [6, 900003, 6000, '2020-09-07'], [7, 900003, -4000, '2020-09-11']]
transactions = pd.DataFrame(data, columns=['trans_id', 'account', 'amount', 'transacted_on']).astype({'trans_id':'Int64', 'account':'Int64', 'amount':'Int64', 'transacted_on':'datetime64[ns]'})
print(users)
print(transactions)

   account     name
0   900001    Alice
1   900002      Bob
2   900003  Charlie
   trans_id  account  amount transacted_on
0         1   900001    7000    2020-08-01
1         2   900001    7000    2020-09-01
2         3   900001   -3000    2020-09-02
3         4   900002    1000    2020-09-12
4         5   900003    6000    2020-08-07
5         6   900003    6000    2020-09-07
6         7   900003   -4000    2020-09-11


In [36]:
import pandas as pd
merge_df = users.merge(transactions, how='left', left_on='account', right_on='account')
# merge_df = users.merge(transactions, how='left', on='account') #當左表和右表的鍵一樣，可以直接寫on=xxx
# print(merge_df)

merge_df = merge_df.groupby(['name'], as_index=False)['amount'].sum().rename(columns = {"amount":"balance"}) #groupby後的結果記得是Series，所以我們要還原欄位
# print(merge_df)
result = merge_df[merge_df['balance']>10000]
print(result)



    name  balance
0  Alice    11000


Pandas 的語法允許你在 groupby() 之後立刻選欄位，相當於：

In [ ]:
merge_df.groupby(['account', 'name']).amount.sum()
# 或
merge_df.groupby(['account', 'name'])['amount'].sum() #聚合後，只取出amount欄位做聚合
# 或
merge_df[['account', 'name', 'amount']].groupby(['account', 'name']).sum()


多重聚合\
1.同欄位聚合

In [ ]:
merge_df.groupby(['account', 'name'], as_index=False).agg({
    'amount': ['sum', 'mean', 'max']
})


2.不同欄位不同聚合

In [ ]:
merged.groupby(['account', 'name'], as_index=False).agg({
    'amount': 'sum',
    'trans_id': 'count'
})


# 1527. Patients With a Condition
Write a solution to find the patient_id, patient_name, and conditions of the patients who have Type I Diabetes. Type I Diabetes always starts with DIAB1 prefix.

In [49]:
data = [[1, 'Daniel', 'YFEV COUGH'], [2, 'Alice', ''], [3, 'Bob', 'DIAB100 MYOP'], [4, 'George', 'ACNE DIAB100'], [5, 'Alain', 'DIAB201'], [6, 'Daniel','SADIAB100']]
patients = pd.DataFrame(data, columns=['patient_id', 'patient_name', 'conditions']).astype({'patient_id':'int64', 'patient_name':'object', 'conditions':'object'})
print(patients)

   patient_id patient_name    conditions
0           1       Daniel    YFEV COUGH
1           2        Alice              
2           3          Bob  DIAB100 MYOP
3           4       George  ACNE DIAB100
4           5        Alain       DIAB201
5           6       Daniel     SADIAB100


In [ ]:
conditions_diab1 = patients[patients['conditions'].str.contains(r'(^|\s)DIAB1')]
print(conditions_diab1)

   patient_id patient_name    conditions
2           3          Bob  DIAB100 MYOP
3           4       George  ACNE DIAB100


/var/folders/s2/gy362bgx7dvct9p2gf3j7t6c0000gn/T/ipykernel_29179/3609920464.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  conditions_diab1 = patients[patients['conditions'].str.contains(r'(^|\s)DIAB1')]


Regex正規表達式：Regular Expression
- r'\bDIAB1' 這個模式會尋找所有 以 'DIAB1' 開頭，並且 'DIAB1' 前面是空白、行首或非字母/數字/底線符號 的字串。
- 正解這邊：str.contains(r'(^|\s)DIAB1')，DIAB1前面開頭（^）只可以是開頭或空格

# 1517. Find Users With Valid E-Mails

Write a solution to find the users who have valid emails.

A valid e-mail has a prefix name and a domain where:

- The prefix name is a string that may contain letters (upper or lower case), digits, underscore '_', period '.', and/or dash '-'. The prefix name must start with a letter.
- The domain is '@leetcode.com'.

In [57]:
data = [[1, 'Winston', 'winston@leetcode.com'], [2, 'Jonathan', 'jonathanisgreat'], [3, 'Annabelle', 'bella-@leetcode.com'], [4, 'Sally', 'sally.come@leetcode.com'], [5, 'Marwan', 'quarz#2020@leetcode.com'], [6, 'David', 'david69@gmail.com'], [7, 'Shapiro', '.shapo@leetcode.com']]
users = pd.DataFrame(data, columns=['user_id', 'name', 'mail']).astype({'user_id':'int64', 'name':'object', 'mail':'object'})
print(users)

   user_id       name                     mail
0        1    Winston     winston@leetcode.com
1        2   Jonathan          jonathanisgreat
2        3  Annabelle      bella-@leetcode.com
3        4      Sally  sally.come@leetcode.com
4        5     Marwan  quarz#2020@leetcode.com
5        6      David        david69@gmail.com
6        7    Shapiro      .shapo@leetcode.com


In [ ]:
regex_pattern = r'^[A-Za-z][A-Za-z0-9_.-]*@leetcode\.com$' # \.跳脫字元，告訴leetcode說我真的要找. # ^開頭，$結尾 #*是量詞，告訴說前面規則可以出現0-n次
result = users[users['mail'].str.contains(regex_pattern, regex=True)]
print(result)

   user_id       name                     mail
0        1    Winston     winston@leetcode.com
2        3  Annabelle      bella-@leetcode.com
3        4      Sally  sally.come@leetcode.com


# 1484. Group Sold Products By The Date
## Hard!


Write a solution to find for each date the number of different products sold and their names.

The sold products names for each date should be sorted lexicographically.

Return the result table ordered by sell_date.


In [59]:
data = [['2020-05-30', 'Headphone'], ['2020-06-01', 'Pencil'], ['2020-06-02', 'Mask'], ['2020-05-30', 'Basketball'], ['2020-06-01', 'Bible'], ['2020-06-02', 'Mask'], ['2020-05-30', 'T-Shirt']]
activities = pd.DataFrame(data, columns=['sell_date', 'product']).astype({'sell_date':'datetime64[ns]', 'product':'object'})
print(activities)

   sell_date     product
0 2020-05-30   Headphone
1 2020-06-01      Pencil
2 2020-06-02        Mask
3 2020-05-30  Basketball
4 2020-06-01       Bible
5 2020-06-02        Mask
6 2020-05-30     T-Shirt


# 183. Customers Who Never Order

In [95]:
data = [[1, 'Joe'], [2, 'Henry'], [3, 'Sam'], [4, 'Max']]
customers = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})
data = [[1, 3], [2, 1]]
orders = pd.DataFrame(data, columns=['id', 'customerId']).astype({'id':'Int64', 'customerId':'Int64'})
print(customers)
print(orders)

   id   name
0   1    Joe
1   2  Henry
2   3    Sam
3   4    Max
   id  customerId
0   1           3
1   2           1


In [117]:
merge_df = customers.merge(orders, how = 'left', left_on='id', right_on='customerId', suffixes=('_customers','_orders'))
merge_df = merge_df[merge_df['id_orders'].isna()]
merge_df_rename = merge_df.rename(columns={'name':'Customer'})
print(merge_df_rename[['Customer']])



  Customer
1    Henry
3      Max


# 196. Delete Duplicate Emails

Write a solution to delete all duplicate emails, keeping only one unique email with the smallest id.

For Pandas users, please note that you are supposed to modify Person in place.

In [118]:
data = [[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'int64', 'email':'object'})
print(person)

   id             email
0   1  john@example.com
1   2   bob@example.com
2   3  john@example.com


In [133]:
# 排序index
sort_index = person.sort_values(by='id', ascending = True) 
# print(sort_index)

# 刪除重複值，並保留最小的index

no_duplicate = sort_index.drop_duplicates(subset=['email'], keep='first') #根據email欄位檢查
print(no_duplicate)

   id             email
0   1  john@example.com
1   2   bob@example.com


# 197. Rising Temperature

Write a solution to find all dates' id with higher temperatures compared to its previous dates (yesterday).

In [137]:
data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype({'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'})
print(weather)

   id recordDate  temperature
0   1 2015-01-01           10
1   2 2015-01-02           25
2   3 2015-01-03           20
3   4 2015-01-04           30


shift: 將所有值向後推一個單位

In [141]:
# 創造一個計算欄位，id2-id1>0就是ans
weather.sort_values(by='recordDate', inplace=True) #inplace會直接取代weather裡面的狀況，我不需要再用其他variable裝結果

#創造2個欄位：prev_temp, prev_date
weather['prev_temp'] = weather['temperature'].shift(1)
weather['prev_date'] = weather['recordDate'].shift(1)
# print(weather)

result = weather[weather['temperature'] > weather['prev_temp']]
result = result.rename(columns={'id':'Id'})
print(result)


   Id recordDate  temperature  prev_temp  prev_date
1   2 2015-01-02           25         10 2015-01-01
3   4 2015-01-04           30         20 2015-01-03


# 511. Game Play Analysis I

## Easy

Write a solution to find the first login date for each player.

In [142]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-05-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})
print(activity)

   player_id  device_id event_date  games_played
0          1          2 2016-03-01             5
1          1          2 2016-05-02             6
2          2          3 2017-06-25             1
3          3          1 2016-03-02             0
4          3          4 2018-07-03             5


In [144]:
# groupby player_id
# min(event_date)

result = activity.groupby(by='player_id', as_index=False)['event_date'].min()
ans = result.rename(columns={'event_date':'first_login'})
print(ans)


   player_id first_login
0          1  2016-03-01
1          2  2017-06-25
2          3  2016-03-02


# 182. Duplicate Emails

## Easy

Write a solution to report all the duplicate emails. Note that it's guaranteed that the email field is not NULL.

  #### 計數
  - len():計算總行數
  - count():計算非遺漏值
  - value_count():計算Series中每個遺漏值出現次數
  - .size():分組後的行數

In [145]:
data = [[1, 'a@b.com'], [2, 'c@d.com'], [3, 'a@b.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'Int64', 'email':'object'})
print(person)

   id    email
0   1  a@b.com
1   2  c@d.com
2   3  a@b.com


In [152]:
# count email出現次數，然後篩出來>1的

email = person[['email']].groupby('email', as_index=False).size()
# print(email)
ans = email[email['size'] >1]
print(ans)

     email  size
0  a@b.com     2


# 1211. Queries Quality and Percentage

## HARD

We define query quality as:

The average of the ratio between query rating and its position.

We also define poor query percentage as:

The percentage of all queries with rating less than 3.

Write a solution to find each query_name, the quality and poor_query_percentage.

Both quality and poor_query_percentage should be rounded to 2 decimal places.

Return the result table in any order.



In [163]:
data = [['Dog', 'Golden Retriever', 1, 5], ['Dog', 'German Shepherd', 2, 5], ['Dog', 'Mule', 200, 1], ['Cat', 'Shirazi', 5, 2], ['Cat', 'Siamese', 3, 3], ['Cat', 'Sphynx', 7, 4]]
queries = pd.DataFrame(data, columns=['query_name', 'result', 'position', 'rating']).astype({'query_name':'object', 'result':'object', 'position':'Int64', 'rating':'Int64'})

print(queries)

  query_name            result  position  rating
0        Dog  Golden Retriever         1       5
1        Dog   German Shepherd         2       5
2        Dog              Mule       200       1
3        Cat           Shirazi         5       2
4        Cat           Siamese         3       3
5        Cat            Sphynx         7       4


# 1378. Replace Employee ID With The Unique Identifier

Write a solution to show the unique ID of each user, If a user does not have a unique ID replace just show null.


In [ ]:
data = [[1, 'Alice'], [7, 'Bob'], [11, 'Meir'], [90, 'Winston'], [3, 'Jonathan']]
employees = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'int64', 'name':'object'})
data = [[3, 1], [11, 2], [90, 3]]
employee_uni = pd.DataFrame(data, columns=['id', 'unique_id']).astype({'id':'int64', 'unique_id':'int64'})

print(employees)
print('----------')
print(employee_uni)

   id      name
0   1     Alice
1   7       Bob
2  11      Meir
3  90   Winston
4   3  Jonathan
----------
   id  unique_id
0   3          1
1  11          2
2  90          3


In [189]:
merge_df = employee_uni.merge(employees, how='right', on='id')
merge_df['unique_id'] = merge_df['unique_id'].fillna('null')
print(merge_df[['unique_id', 'name']])

  unique_id      name
0      null     Alice
1      null       Bob
2       2.0      Meir
3       3.0   Winston
4       1.0  Jonathan


# 1633. Percentage of Users Attended a Contest

Write a solution to find the percentage of the users registered in each contest rounded to two decimals.

Return the result table ordered by percentage in descending order. In case of a tie, order it by contest_id in ascending order.


In [191]:
data = [[6, 'Alice'], [2, 'Bob'], [7, 'Alex']]
users = pd.DataFrame(data, columns=['user_id', 'user_name']).astype({'user_id':'Int64', 'user_name':'object'})
data = [[215, 6], [209, 2], [208, 2], [210, 6], [208, 6], [209, 7], [209, 6], [215, 7], [208, 7], [210, 2], [207, 2], [210, 7]]
register = pd.DataFrame(data, columns=['contest_id', 'user_id']).astype({'contest_id':'Int64', 'user_id':'Int64'})

print(users)
print(register)

   user_id user_name
0        6     Alice
1        2       Bob
2        7      Alex
    contest_id  user_id
0          215        6
1          209        2
2          208        2
3          210        6
4          208        6
5          209        7
6          209        6
7          215        7
8          208        7
9          210        2
10         207        2
11         210        7


In [202]:
# 先merge table
# 計算percentage: 參加人數 / 總人數

merge_df = register.merge(users, how = 'left', on = 'user_id')
merge_df = merge_df[['contest_id','user_name']]
# print(merge_df)

total_user = merge_df['user_name'].nunique()
# print(total_user)

grouped = merge_df.groupby('contest_id', as_index=False)['user_name'].size().sort_values('size', ascending=False)
grouped['percentage'] = ((grouped['size'] / (total_user)) * 100).round(2)

resulted_ans = grouped.sort_values(by=['percentage', 'contest_id'], ascending=[False, True])
print(resulted_ans[['contest_id', 'percentage']])


   contest_id  percentage
1         208      100.00
2         209      100.00
3         210      100.00
4         215       66.67
0         207       33.33


# 2877. Create a DataFrame from List

List轉DataFrame，並創造欄位標題


In [207]:
student_data = [[1, 15],[2, 11],[3, 11],[4, 20]]
result = pd.DataFrame(student_data, columns=['student_id','age'])
print(result)



   student_id  age
0           1   15
1           2   11
2           3   11
3           4   20


# 1873. Calculate Special Bonus
## HARD

Write a solution to calculate the bonus of each employee. The bonus of an employee is 100% of their salary if the ID of the employee is an odd number and the employee's name does not start with the character 'M'. The bonus of an employee is 0 otherwise.

Return the result table ordered by employee_id.


In [224]:
data = [[2, 'Meir', 3000], [3, 'Michael', 3800], [7, 'Addilyn', 7400], [8, 'Juan', 6100], [9, 'Kannon', 7700]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'salary']).astype({'employee_id':'int64', 'name':'object', 'salary':'int64'})
print(employees)

   employee_id     name  salary
0            2     Meir    3000
1            3  Michael    3800
2            7  Addilyn    7400
3            8     Juan    6100
4            9   Kannon    7700


兩個條件：name如果非M開頭，以及ID為基數，則bonus = salary

布林值篩選，用np.where('條件','滿足時的值','不滿足時的值')

### np.where寫法

In [226]:
import pandas as pd
import numpy as np

condition_name = (employees['name'].str[0].str.upper() != 'M')
# print(condition_name)

condition_id = (employees['employee_id'] % 2 != 0)
# print(condition_id)

condition = condition_name & condition_id

employees['bonus'] = np.where (condition, employees['salary'], 0)
print(employees)


   employee_id     name  salary  bonus
0            2     Meir    3000      0
1            3  Michael    3800      0
2            7  Addilyn    7400   7400
3            8     Juan    6100      0
4            9   Kannon    7700   7700


# 1741. Find Total Time Spent by Each Employee

Write a solution to calculate the total time in minutes spent by each employee on each day at the office. 

Note that within one day, an employee can enter and leave more than once. 

The time spent in the office for a single entry is out_time - in_time.


In [231]:
data = [['1', '2020-11-28', '4', '32'], ['1', '2020-11-28', '55', '200'], ['1', '2020-12-3', '1', '42'], ['2', '2020-11-28', '3', '33'], ['2', '2020-12-9', '47', '74']]
employees = pd.DataFrame(data, columns=['emp_id', 'event_day', 'in_time', 'out_time']).astype({'emp_id':'Int64', 'event_day':'datetime64[ns]', 'in_time':'Int64', 'out_time':'Int64'})
print(employees)

   emp_id  event_day  in_time  out_time
0       1 2020-11-28        4        32
1       1 2020-11-28       55       200
2       1 2020-12-03        1        42
3       2 2020-11-28        3        33
4       2 2020-12-09       47        74


In [238]:
employees['total_time'] = employees['out_time'] - employees['in_time']
# print(employees)

result = employees.groupby(by=['event_day','emp_id'], as_index=False)['total_time'].sum()
rename_ans = result.rename(columns={'event_day':'day'})
print(rename_ans)

         day  emp_id  total_time
0 2020-11-28       1         173
1 2020-11-28       2          30
2 2020-12-03       1          41
3 2020-12-09       2          27


# 1693. Daily Leads and Partners

For each date_id and make_name, find the number of distinct lead_id's and distinct partner_id's.

Return the result table in any order.

In [240]:
data = [['2020-12-8', 'toyota', 0, 1], ['2020-12-8', 'toyota', 1, 0], ['2020-12-8', 'toyota', 1, 2], ['2020-12-7', 'toyota', 0, 2], ['2020-12-7', 'toyota', 0, 1], ['2020-12-8', 'honda', 1, 2], ['2020-12-8', 'honda', 2, 1], ['2020-12-7', 'honda', 0, 1], ['2020-12-7', 'honda', 1, 2], ['2020-12-7', 'honda', 2, 1]]
daily_sales = pd.DataFrame(data, columns=['date_id', 'make_name', 'lead_id', 'partner_id']).astype({'date_id':'datetime64[ns]', 'make_name':'object', 'lead_id':'Int64', 'partner_id':'Int64'})

print(daily_sales)

     date_id make_name  lead_id  partner_id
0 2020-12-08    toyota        0           1
1 2020-12-08    toyota        1           0
2 2020-12-08    toyota        1           2
3 2020-12-07    toyota        0           2
4 2020-12-07    toyota        0           1
5 2020-12-08     honda        1           2
6 2020-12-08     honda        2           1
7 2020-12-07     honda        0           1
8 2020-12-07     honda        1           2
9 2020-12-07     honda        2           1


#### 聚合操作：agg(新名稱 = (舊名稱,函數))，搭配groupby()聚合
- df['salary'].agg('mean')
- df[['col1', 'col2']].agg('sum')
- groupby().agg() :df.groupby('city').agg(TotalSum=('price', 'sum'),AveragePrice=('price', 'mean'))



In [255]:
# groupby date_id, make_name，透過nunique去算或size

group_df = daily_sales.groupby(by=['date_id', 'make_name'], as_index=False).agg(unique_leads=('lead_id', 'nunique'), unique_partners=('partner_id', 'nunique'))
group_df = group_df.sort_values(by = ['make_name', 'unique_partners'], ascending=[False,False])
print(group_df)

     date_id make_name  unique_leads  unique_partners
3 2020-12-08    toyota             2                3
1 2020-12-07    toyota             1                2
0 2020-12-07     honda             3                2
2 2020-12-08     honda             2                2


# 584. Find Customer Referee
### Easy

Find the names of the customer that are either:

- referred by any customer with id != 2.
- not referred by any customer.
Return the result table in any order.

In [264]:
data = [[1, 'Will', None], [2, 'Jane', None], [3, 'Alex', 2], [4, 'Bill', None], [5, 'Zack', 1], [6, 'Mark', 2]]
customer = pd.DataFrame(data, columns=['id', 'name', 'referee_id']).astype({'id':'Int64', 'name':'object', 'referee_id':'Int64'})
print(customer)

   id  name  referee_id
0   1  Will        <NA>
1   2  Jane        <NA>
2   3  Alex           2
3   4  Bill        <NA>
4   5  Zack           1
5   6  Mark           2


In [267]:
not_referred = customer[(customer['referee_id'].isna()) | (customer['referee_id'] != 2)]
print(not_referred['name'])

0    Will
1    Jane
3    Bill
4    Zack
Name: name, dtype: object


# 607. Sales Person

Write a solution to find the names of all the salespersons who did not have any orders related to the company with the name "RED".

Return the result table in any order.

In [268]:
data = [[1, 'John', 100000, 6, '4/1/2006'], [2, 'Amy', 12000, 5, '5/1/2010'], [3, 'Mark', 65000, 12, '12/25/2008'], [4, 'Pam', 25000, 25, '1/1/2005'], [5, 'Alex', 5000, 10, '2/3/2007']]
sales_person = pd.DataFrame(data, columns=['sales_id', 'name', 'salary', 'commission_rate', 'hire_date']).astype({'sales_id':'Int64', 'name':'object', 'salary':'Int64', 'commission_rate':'Int64', 'hire_date':'datetime64[ns]'})
data = [[1, 'RED', 'Boston'], [2, 'ORANGE', 'New York'], [3, 'YELLOW', 'Boston'], [4, 'GREEN', 'Austin']]
company = pd.DataFrame(data, columns=['com_id', 'name', 'city']).astype({'com_id':'Int64', 'name':'object', 'city':'object'})
data = [[1, '1/1/2014', 3, 4, 10000], [2, '2/1/2014', 4, 5, 5000], [3, '3/1/2014', 1, 1, 50000], [4, '4/1/2014', 1, 4, 25000]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'com_id', 'sales_id', 'amount']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'com_id':'Int64', 'sales_id':'Int64', 'amount':'Int64'})


print(sales_person)
print(company)
print(orders)

   sales_id  name  salary  commission_rate  hire_date
0         1  John  100000                6 2006-04-01
1         2   Amy   12000                5 2010-05-01
2         3  Mark   65000               12 2008-12-25
3         4   Pam   25000               25 2005-01-01
4         5  Alex    5000               10 2007-02-03
   com_id    name      city
0       1     RED    Boston
1       2  ORANGE  New York
2       3  YELLOW    Boston
3       4   GREEN    Austin
   order_id order_date  com_id  sales_id  amount
0         1 2014-01-01       3         4   10000
1         2 2014-02-01       4         5    5000
2         3 2014-03-01       1         1   50000
3         4 2014-04-01       1         4   25000


In [276]:
# SalesPerson table-> Orders table-> Company table
merge_df1 = sales_person.merge(orders, how = 'left', on = 'sales_id')
merge_df2 = merge_df1.merge(company, how='left', on = 'com_id')
result = merge_df2[(merge_df2['name_y'] !='RED')]
print(result[['name_x']])

# merge_df2

  name_x
1    Amy
2   Mark
3    Pam
5   Alex
